<a href="https://colab.research.google.com/github/yunxianding/CSC120-A2/blob/master/Reverse_Motion_Diagrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to computational physics:  "Motion Diagrams in Reverse"

In the early part of the course, we've used "motion diagrams" to help us understand and visualize the velocity and acceleration associated with a moving object.  

A motion diagram starts with the given locations of the object at a number of equally-spaced moments in time.  We can draw arrows from one dot to the next to represent the displacement during each time interval and, because the average velocity during each time interval is proportional to the displacement, i.e.,

$$ \vec{v} = \dfrac{\Delta \vec{r}}{\Delta t}, $$

those same arrows can also represent the velocities.  Then, the velocities associated with two successive time intervals can be graphically subtracted to give the change in velocity $\Delta\vec{v}$.  And since the acceleration is just proportional to the change in velocity, i.e.,

$$ \vec{a} = \dfrac{\Delta \vec{v}}{\Delta t},$$

the same arrow we drew to represent the change in velocity can also represent the acceleration.  In short, from the given set of dots (representing the object's positions at a bunch of times) we can figure out the velocity and acceleration at (or, more precisely, between) all those times.

Our goal today is to explore the use of these same ideas and relationships to "go backwards" -- *from* given information about the acceleration of an object *to* a set of dots representing its position at various times -- and, in particular, to learn how computers can help us implement the needed algorithms and visualize the results.  

# 1. Review Example

Let's jump in by quickly reviewing motion diagrams in the context of an example.  

Travis propped a board on a brick to make a ramp and rolled a pool ball upward from the bottom.  On roughly the 17th try, he managed to start it so it went up nearly to the top of the ramp (but without falling off), turned around, and rolled back down.  Here's a video:

[![Ball on Ramp](https://drive.google.com/uc?export=view&id=1yhZQ--XXVbP1ymvCBZd61J3Xyk2AKilS)](https://www.youtube.com/watch?v=TfoUY20L79E&ab_channel=TravisNorsen "Ball on Ramp")

As we saw last week, it's possible to import a video like this into LoggerPro, go through frame by frame, and mark the position of the object at a bunch of equally-spaced moments in time.  

Doing this for the ball rolling up and then back down resulted in the following image (with numbers added so it's clear what order the dots come in):

<img src="https://drive.google.com/uc?export=view&id=1ma9Jw6si6Dp4BEaGl2MFWvSnkCeoazn5" width="1000">

And then with the dots given, we can go through the hopefully-familiar process of drawing arrows to represent the velocity (green) and acceleration (red):



<img src="https://drive.google.com/uc?export=view&id=1e-YawgSBfYmyC8ZCSGONCVEMko2wrhWW" width="1000">

Sorry that image is a bit cluttered with arrows, but I wanted to show (for points 2 through 9) how I figured out what the red acceleration arrows look like, hence the vector subtraction diagrams around the perimeter.  Anyway, we could summarize the results here by saying that, if we take the x-axis to be parallel to the ramp with its positive direction going uphill (as shown in the above image), graphs of the x coordinate, the (x-component of the) velocity, and the (x-component of the) acceleration look something like this:

<center>
<img src="https://drive.google.com/uc?export=view&id=1Gs1Nn4pMCnUmR3W2FSpDDmGQPp-KJkny" width="600">
</center>

Good.  Hopefully all of this is clear and familiar.  The main point is that here we used the given data about the positions of the ball at various times to figure out that the acceleration points downward along the ramp and with a seemingly pretty constant magnitude, i.e., that the x-component of the acceleration has a constant negative value.  

## Do this:

* Pick two or three of the numbered dots and, as a group, at the whiteboard, work through the graphical vector subtraction process to see what the acceleration looks like.  Basically check the work I did/showed in the image, but maybe labeling things more clearly, to make sure you understand how I got what I got!


## Then some questions for quick discussion at your tables

* Are the (green) velocity vectors in the motion diagram above instantaneous velocities, or average velocities?  

* Should you think of them as associated with the moments (e.g., moment 2 or moment 5) or the time-intervals between moments (e.g., the 2 $\rightarrow$ 3 interval or the 5 $\rightarrow$ 6 interval)?

* Same questions for the (red) acceleration vectors...

# 2. Going the other way

Now let's think through how, if we were given that (say) the ball moved along the x-axis with a constant negative acceleration, we could reconstruct the dots representing its position at various times.  We'll do this first just in terms of pictures.  Then we'll step back and think more carefully about what's happening mathematically/quantitatively.  And then finally we'll turn to writing some computer code to have the computer do the calculations for us!  

Here's how this works in terms of the ball-on-ramp example.  Suppose we know that at moment 1 the ball's location is given by the dot in the following image, its velocity at moment 1 is represented by the green arrow, and the constant acceleration is given by the red arrow:

<img src="https://drive.google.com/uc?export=view&id=1EYxtG5DSNQ82de7tbqNvIpnZLVW12z9U" width="1000">


Then we can use the given velocity at moment 1 to figure out where the next dot should be, i.e., what the position should be at moment 2... and we can use the given velocity at moment 1 and the known acceleration to figure out what the velocity will be at moment 2:


<img src="https://drive.google.com/uc?export=view&id=1WRVJkL-88o5AW9gZ1tufUBkfEiaNHfKw" width="1000">

The crucial point here is that now we are in exactly the same position with respect to moment 2, that we were in with respect to moment 1 before:  we know the ball's location and its velocity at that moment (and we know what the acceleration is because we're assuming it just stays constant).  And so we could do the same procedure again, using the now-known position and velocity at moment 2 (and the acceleration) to find the position and velocity at moment 3.  And then we could do it again, and again, and again... stepping our way forward in time to generate as many dots as we might want.

This repeating algorithm is the kind of thing computers are really good at implementing for us (more quickly and efficiently and accurately than we could do it by hand with pictures).  We'll turn to that in a second, but first let's quickly fill in a couple of mathematical details.  

What's really going on when we use the position and velocity at one moment (where those things are known) to figure out the position at the next moment?  We're actually just using the definition of (average) velocity mentioned above, namely

$$\vec{v} = \dfrac{\Delta \vec{r}}{\Delta t},$$

to calculate the displacement:

$$ \Delta \vec{r} = \vec{v} \, \Delta t.$$

Or, remembering that the displacement (say, during the time interval between "moment n" and "moment n+1") is just $\vec{r}_{n+1} - \vec{r}_n$, we can rearrange the above equation to read:

$$ \vec{r}_{n+1} = \vec{r}_n + \vec{v}_n \, \Delta t.$$

We can think of this as an "update rule":  if we know the position ($\vec{r}_n$) and velocity ($\vec{v}_n$) at moment n, we can use them to calculate the new position at the next moment ($\vec{r}_{n+1}$).  

There is one subtlety that we should pause to consider.  In this section, we've been thinking of velocities as associated with specific moments.  For example, we got started by assuming we were given $\vec{v}_1$, the velocity of the ball *at* moment 1, and then, by going through this updating procedure, we worked our way up to knowing the velocity $\vec{v}_n$ *at* moment n.  That is, the velocities we've been working with here are *instantaneous velocities*.

But, strictly speaking, the last few equations are only true if the velocities are *average velocities* -- not the velocities *at* specific moments, but the average velocities for the time intervals *between* moments.  

Does this make a difference?  Yes!  If the velocity is changing (and... if there's acceleration... it is!) the instantaneous velocity at some moment will not be the same as the average velocity during a time interval starting at that moment.  But... if the velocity isn't changing very fast... or, equivalently, if we consider $\Delta t$, the time interval between one moment and the next, to be very small, then the velocity won't change very much during the time interval, and the instantaneous velocity at the beginning of the time interval will be a very good approximation to the average velocity during the time interval.  

Note also that this same discussion applies also, in exactly the same way, to the other part of our algorithm:  using the given acceleration to "update" the velocity.  We can rearrange the definition of (average!) acceleration

$$\vec{a} = \dfrac{\Delta \vec{v}}{\Delta t}$$

into an "update rule" for the velocity:

$$\vec{v}_{n+1} = \vec{v}_n + \vec{a}_n \, \Delta t.$$

Strictly speaking this is exactly true only if $\vec{a}_n$ is the *average* acceleration during the period between moment n and moment n+1.  If (as in the example we've been working with) the acceleration is *constant*, then this average acceleration will be the same as the instantaneous acceleration at moment n.  So for this particular example, our "update rule" for velocities will be exact.  But in some other examples we'll consider later, the acceleration may not be constant, and then our update rule will be only approximately true... but the approximation will be better the smaller we make $\Delta t$.  

To summarize, the algorithm we're developing here is not perfectly accurate.  But we can make it as accurate as we want by making $\Delta t$ small.  So there's a trade-off between accuracy and work.  If (say) we want to figure out where the ball is after one second, we could get that answer by taking a single step with $\Delta t = 1 \, \text{sec}$.  But that won't be very accurate!  We could make it more accurate by taking $\Delta t = 0.2 \, \text{sec}$, but then we'd have to take 5 steps forward in time.  Or by taking 50 steps with $\Delta t = 0.02 \, \text{sec}$ we could find the location of the ball after one second even *more* accurately.  You get the idea.  The point is, to achieve good accuracy, we're going to want to take little tiny baby steps (instead of just a few giant leaps).  But that means taking *lots* of steps, which will get really tedious and labor-intensive if we're doing it all by hand.  

So in a minute we'll start figuring out how to farm the labor out to the computer!  

## But first:  

* In the images above I drew the first step of the position and velocity updating algorithm -- using the position and velocity at time 1 (and the given constant acceleration) to determine the position and velocity at time 2.  Reproduce this with your group on the white boards, and then work through another few steps so you really get the hang of the algorithm.

* Now suppose you start that process over again from the beginning (i.e., knowing the position and velocity at time 1), but with a smaller $\Delta t$.  Draw the first few steps of the process again.  Note that if you were using the same arrows before to represent both $\vec{v}$ and $\Delta \vec{r}$ vectors, you'll need to do something slightly different now for the new, smaller value of $\Delta t$!

* Which version of the process is better?  The original one with the bigger $\Delta t$, or the newer one with the smaller $\Delta t$?  Discuss.

Check in with an instructor or LA to share your diagrams and thoughts.  




# 3. Implementing our algorithm with python code

So far in this document you've just been reading "text cells" (with pre-written text, equations, pictures, etc.).  But the nice thing about this type of document, which is called a Jupyter notebook, is that we can also have "code cells" where we write, run, tweak, and re-run computer code in (e.g.) the programming language Python.  

If you've had some experience with computer programming in the past (whether with Python or some other language), this should be pretty comfortable and you'll probably be able to jump right into understanding and playing with the computer code below.  If you haven't, it'll take a bit more time and effort to get comfortable, but don't worry -- this isn't a computer programming class and we'll never ask you to write code from scratch.  Instead, we'll give you working code that you can just use as a tool even if you're not completely comfortable with all of the details under the hood.  That said, you can learn a lot by looking under the hood even if you're not sure exactly how everything works.  So even if computer code is a little unfamiliar and scary, look through it and try to understand what it's doing and you'll be on the path to making it familiar and unscary!

Anyway, here is some code that implements 5 steps of the "update rule" algorithm we've been discussing.  Look through it first and try to predict what it will do when you run it.  Discuss with your group.  Then run it (by clicking the little "play" button in the upper left corner of the cell) and keep reading below...

In [ ]:

# Code to implement 5 steps of the algorithm discussed above, for the ball-rolling-on-a-ramp example
#  where the (x-component of the) acceleration has a constant negative value.
# Note that bits of code, like this, which come after a "#" sign are "comments" (not actual code).
# Reading them should help you follow what's going on in the actual code.


# import some packages (i.e., standard, pre-existing chunks of code) that we'll use for standard math operations and plotting

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets import interact
from ipywidgets import FloatSlider,Layout,IntSlider


# Define constants and initial values

a = -0.8    # acceleration in m/s^2
x_0 = 0.0   # initial value of x
v_0 = 1.0   # initial velocity in m/s

Dt = 0.2  # time step "Delta t" in seconds


# Let's create "arrays" (i.e., lists) to store the values of t, x, and v at our 6 moments.
# By the way, we want 6 moments so we can take 5 steps...

t = np.zeros(6)  # creates an array with elements t[0], t[1], t[2], t[3], t[4], and t[5], all set to zero
x = np.zeros(6)  # same for x...
v = np.zeros(6)  # same for v

# Note that in python the first element in a list is the "zeroth" element -- e.g., the first element of the array called "t"
# is not t[1], but rather t[0].  So our 6 moments will be called 0, 1, 2, 3, 4, and 5...


# Fill in the initial values:

x[0] = x_0
v[0] = v_0

# t[[0]] was already set to zero!

# Now we're ready to implement the first step -- from "moment 0" to "moment 1"

t[1] = t[0] + Dt         # update the time

x[1] = x[0] + v[0]*Dt    # update rule for the position

v[1] = v[0] + a*Dt       # update rule for the velocity


# And now we can take another step:

t[2] = t[1] + Dt
x[2] = x[1] + v[1]*Dt    # update rule for the position
v[2] = v[1] + a*Dt       # update rule for the velocity


# And another:

t[3] = t[2] + Dt
x[3] = x[2] + v[2]*Dt    # update rule for the position
v[3] = v[2] + a*Dt       # update rule for the velocity


# And another:

t[4] = t[3] + Dt
x[4] = x[3] + v[3]*Dt    # update rule for the position
v[4] = v[3] + a*Dt       # update rule for the velocity


# And finally the last one:

t[5] = t[4] + Dt
x[5] = x[4] + v[4]*Dt    # update rule for the position
v[5] = v[4] + a*Dt       # update rule for the velocity


# print the x values just so there's some output...

print(x)


Did it do what you expected?  Discuss (with your group or an instructor/LA) any questions you have about what's happening!

Then when you're ready, you can run the next 3 code cells that make various kinds of plots of the results...

In [ ]:
# This code makes a plot that shows dots representing the positions at our 5 moments, like in a motion diagram.

y = np.zeros(6)  #create an array of y values (all set to zero) so we can plot dots in the xy plane...

plt.figure(figsize=(10,2))  #create a figure with a certain size
plt.plot(x,y,'k.')        # plot x -- the list of x coordinates we generated above -- vs. zero
plt.title("Positions at our 6 moments")  # give the plot a title
plt.xlabel("x (in meters)")  # label for the horizontal axis
plt.ylabel("y (in meters)")  # label for the vertical axis
plt.axis([-.2,.8,-.2,.2])   # set the range/scale on the x and y axes
plt.show()                   # draw the plot

In [ ]:
# Here is a plot of x vs t

plt.figure(figsize=(7,4))  #create a figure with a certain size
plt.plot(t,x,'b.')          # plot x -- the list of x coordinates we generated above -- vs. zero
plt.title("x vs t")  # give the plot a title
plt.xlabel("t (in sec)")  # label for the horizontal axis
plt.ylabel("x (in meters)")  # label for the vertical axis
#plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
plt.show()                   # draw the plot

In [ ]:
# Here is a plot of v vs t

plt.figure(figsize=(7,4))  #create a figure with a certain size
plt.plot(t,v,'g.')          # plot x -- the list of x coordinates we generated above -- vs. zero
plt.title("v vs t")  # give the plot a title
plt.xlabel("t (in sec)")  # label for the horizontal axis
plt.ylabel("v (in m/s)")  # label for the vertical axis
#plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
plt.show()                   # dr

So far all of this should strike you as more or less what we expected.  The motion diagram looks like the first few dots of our motion diagram for the ball on the ramp (at least if you tilt your head... I couldn't figure out how to make python display this graph rotated, but remember if we're thinking of this as the ball-on-a-ramp scenario, the x-axis is tilted, parallel to the ramp). The graph of x vs t looks like the beginning of the one I sketched up above... and so does the graph of v vs t.  

Anyway, the code above works great and is relatively easy to understand.  But it's a bit awkward and inelegant.  

The next code cell does exactly the same thing (including making all 3 graphs in one nicely-organized output), but is much more efficient and flexible.  Take a look through it, then run it to confirm that it works, but then go back, fiddle with the values of "Nsteps" and "Dt", and re-run the code, to see how things come out.  

In [ ]:

# Set values for the number of steps to take, and the timestep size

Nsteps = 10  # number of steps
Dt = 0.2    # temporal size of steps


# Define constants and initial values

a = -0.8    # acceleration in m/s^2
x_0 = 0.0   # initial value of x
v_0 = 1.0   # initial velocity in m/s


# Create arrays to fill in

t = np.zeros(Nsteps+1)
x = np.zeros(Nsteps+1)
v = np.zeros(Nsteps+1)

# Fill in initial values

x[0] = x_0
v[0] = v_0


# Now do all the steps using a loop:


for n in range(Nsteps):
    t[n+1] = t[n] + Dt
    x[n+1] = x[n] + v[n]*Dt     # update rule for y:  the new y is the old y plus (the old) v times the timestep
    v[n+1] = v[n] + a*Dt        # update rule for v...



y = np.zeros(Nsteps+1)  #set all y coordinates to zero (we'll use this again in the plot...)



# and now make a multi-part plot...

plt.figure(figsize=(14,7))
gridspec.GridSpec(4,2)

# motion diagram figure
plt.subplot2grid((4,2),(0,0),colspan=2)
plt.plot(x,y,'k.')        # plot x -- the list of x coordinates we generated above -- vs. zero
plt.title("Positions...")  # give the plot a title
plt.xlabel("x (in meters)")  # label for the horizontal axis
plt.ylabel("y (in meters)")  # label for the vertical axis
plt.axis([-.2,1.2,-.2,.2])   # set the range/scale on the x and y axes
#plt.show()                   # draw the plot

# x vs t graph
plt.subplot2grid((4,2),(2,0),rowspan=2)
plt.plot(t,x,'b.')          # plot x -- the list of x coordinates we generated above -- vs. zero
plt.title("x vs t")  # give the plot a title
plt.xlabel("t (in sec)")  # label for the horizontal axis
plt.ylabel("x (in meters)")  # label for the vertical axis
#plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
#plt.show()                   # draw the plot


# v vs t graph
plt.subplot2grid((4,2),(2,1),rowspan=2)
plt.plot(t,v,'g.')          # plot x -- the list of x coordinates we generated above -- vs. zero
plt.title("v vs t")  # give the plot a title
plt.xlabel("t (in sec)")  # label for the horizontal axis
plt.ylabel("v (in m/s)")  # label for the vertical axis
#plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
#plt.show()                   # draw the plot

plt.show()



So, as you can see, my code takes 10 steps with $\Delta t = 0.2$ seconds, so it tracks the motion up until $t = 2.0$ seconds.  

Notice that, at some time a little before $t = 2.0$ seconds, the velocity crosses through zero and the position has a local maximum value.  As accurately as you can estimate from the graphs, when exactly does this happen?

Now go back and tweak the code and ask it to instead take 20 steps with $\Delta t = 0.1$ seconds.  Re-run it.  Does the local maximum in the position (i.e., the "turning point" where v=0) happen at exactly the same time as before, or is it a little different?  

If (as expected) you can see that it's a little different, why do you think it's different?  Which value do you think is more accurate? Discuss with your group (and an instructor/LA if needed) and achieve clarity and consensus before continuing.





# 4. Exploring the numerical accuracy

When you're ready to proceed, take a look at, and then run, the following code cell.  It basically does the same thing as the last one, but with the following improvements:

* Instead of setting Nsteps and Dt, you can set Nsteps and tfinal and it just calculates Dt automatically.

* The output of the code cell will be an "interactive widget" that allows you to control Nsteps and tfinal with slider bars (instead of having to go into the code, type new values, and re-run the cell).

* In addition to plotting the x and v values generated by our numerical algorithm, the x-vs-t and v-vs-t graphs also include graphs of what we expect "analytically" (i.e., using the constant acceleration kinematics equations developed in Chapter 2 of the text).  This allows you to see, very easily, that and how the accuracy of our algorithm changes as you increase Nsteps (and thereby make Dt smaller) as discussed previously. I also asked it to calculate the "error" -- the difference between x(tfinal) as calculated from our numerical algorithm and x(tfinal) as calculated from the analytic formulas -- and display that in a text box on the x-vs-t graph so it's really easy to see a quantitative measure of the accuracy of the numerical algorithm.

It'll probably be a little harder to understand exactly how all of this code works, but by all means take a look and understand what you can.  But then feel free to just run it and play around with the interactive widget.


In [ ]:


# First we define a function called "makegraph" that takes values of Nsteps and tfinal as inputs, cranks through the whole calculation,
# and outputs the plots...


def makegraph(Nsteps,tfinal):

  Dt=tfinal/(Nsteps)

  # Define constants and initial values
  a = -0.8    # acceleration in m/s^2
  x_0 = 0.0   # initial value of x
  v_0 = 1.0   # initial velocity in m/s

  # Create arrays to fill in
  t = np.zeros(Nsteps+1)
  x = np.zeros(Nsteps+1)
  v = np.zeros(Nsteps+1)

  # Fill in initial values
  x[0] = x_0
  v[0] = v_0

  # Now do all the steps using a loop:
  for n in range(Nsteps):
    t[n+1] = t[n] + Dt
    x[n+1] = x[n] + v[n]*Dt     # update rule for y:  the new y is the old y plus (the old) v times the timestep
    v[n+1] = v[n] + a*Dt        # update rule for v...

  y = np.zeros(Nsteps+1)  #set all y coordinates to zero (we'll use this again in the plot...)

  # Calculate analytic values for comparison
  times=np.linspace(0,tfinal,1000)
  x_analytic = x_0 + v_0*times + 0.5*a*times*times
  v_analytic = v_0 + a*times

  # Calculate the "error" -- the difference between the values of "x" and "x_analytic" at tfinal
  error = x[Nsteps]-x_analytic[999]


  # and now make a multi-part plot...
  plt.figure(figsize=(14,7))
  gridspec.GridSpec(4,2)

  # motion diagram figure
  plt.subplot2grid((4,2),(0,0),colspan=2)
  plt.plot(x,y,'k.')        # plot x -- the list of x coordinates we generated above -- vs. zero
  plt.title("Positions...")  # give the plot a title
  plt.xlabel("x (in meters)")  # label for the horizontal axis
  plt.ylabel("y (in meters)")  # label for the vertical axis
  plt.axis([-.2,1.2,-.2,.2])   # set the range/scale on the x and y axes
  #plt.show()                   # draw the plot

  # x vs t graph
  plt.subplot2grid((4,2),(2,0),rowspan=2)
  plt.plot(t,x,'b.',label="numerical")          # plot x -- the list of x coordinates we generated above -- vs. zero
  plt.plot(times,x_analytic,'k-',label="analytic")  #plot also the analytic x vs t
  plt.title("x vs t")  # give the plot a title
  plt.xlabel("t (in sec)")  # label for the horizontal axis
  plt.ylabel("x (in meters)")  # label for the vertical axis
  plt.legend()
  text = "error = {:.5f}".format(error) #text for the text box
  plt.text(.6,.2,text,fontsize=12,verticalalignment='center',bbox=dict(boxstyle='round',facecolor='wheat',alpha=0.5))
  #plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
  #plt.show()                   # draw the plot


  # v vs t graph
  plt.subplot2grid((4,2),(2,1),rowspan=2)
  plt.plot(t,v,'g.',label="numerical")          # plot x -- the list of x coordinates we generated above -- vs. zero
  plt.plot(times,v_analytic,'k-',label="analytic")  #plot also the analytic v vs t
  plt.title("v vs t")  # give the plot a title
  plt.xlabel("t (in sec)")  # label for the horizontal axis
  plt.ylabel("v (in m/s)")  # label for the vertical axis
  plt.legend()
  #plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
  #plt.show()                   # draw the plot

  plt.show()


# Now create an interactive widget with slider bars to give input to the function makegraph

interact(makegraph,
         Nsteps=IntSlider(min=1, max=300, step=1, value=5, layout=Layout(width='80%')),
         tfinal=FloatSlider(min=0.1, max=5, step=.1, value=1.0, readout_format='.1f', layout=Layout(width='80%'))
        );

## Explore:

* Set tfinal to 2.5 seconds -- roughly the time it takes the ball to roll up, turn around, and roll back down to about where it started.  With Nsteps = 5, what is the "error" (i.e., the difference between what x should be at 2.5 seconds according to the analytic formulas, and what our approximate numerical algorithm comes up with for x at 2.5 seconds)?

* What happens to the value of "error" as you increase the value of Nsteps?  How many steps (from t=0 to t=2.5 seconds) are required to produce an error less than 0.1 meters?

* How many steps are required to produce an error less than 0.01 meters?

* *Why* does the numerical algorithm become more accurate as Nsteps is increased?  Discuss with your group until you reach a shared understanding, and then share your explanation with an instructor or LA.

# 5. A new example

So far we've just been working with the ball-on-a-ramp type example where the acceleration is simply constant.  Of course, as you might have been thinking, this way of doing things that we've been developing -- making "motion diagrams in reverse" using the computer to implement our "update rules" numerically -- is completely pointless for this example:  if the acceleration is constant, we already have the nice clean analytic formulas for what x(t) and v(t) will be!  

So really you should understand all of the above discussion as developing this idea in the context of an example where we already know what the answer should look like, so we can assess whether the novel approach is doing what it should be doing.  But the *point* of the approach is to use it for different examples, where the acceleration is not constant -- but is maybe some complicated function of time and/or the position and/or the velocity -- and where we therefore *don't have* pre-made analytic kinematics equations, i.e., where we *don't already know* exactly what should happen.  Then we can use our numerical algorithm to figure out what'll happen.  

Let's explore the idea with the following example.  Suppose a cart with perfectly frictionless wheels starts at $x=0$ with an initial velocity $v_0$ in the positive x-direction (which, unlike the ball-on-a-ramp example above, we now take to be perfectly horizontal).  If that were the whole story, the cart would just glide at constant velocity in a straight line forever.  But suppose the cart is equipped with a "sail" which gives rise to some air drag.  Suppose in particular that this air drag gives the cart an acceleration, opposite the direction it is moving, whose magnitude is proportional to its velocity at each moment.  That is, suppose

$$a = - k \, v$$

for some (positive) constant $k$.  

How will the cart move?  Go to the white boards with your group, and try to implement our "motion diagrams in reverse" algorithm by hand, qualitatively, like you did before for the case where the acceleration was constant.  See if you can come up with the motion diagram picture and also x-vs-t and v-vs-t graphs.  

Check in with an instructor or LA when you're done (and/or if/when you get stuck!).  

Then you can proceed to look at, try to understand, and finally run the following block of code and compare its output to what you came up with.  

In [ ]:

# define the acceleration as a function of the velocity
# we'll use the value k = 0.50 (m/s^2)/(m/s) = 0.50 s^{-1}.
# this means that an object moving with v = 10.0 m/s, will have an acceleration a = -5.0 m/s^2.
# But of course you can always change this if you want to play around...

k = 0.50  # constant from "a = -k*v" in inverse seconds

def a(v):
  a = -k*v
  return a


# Now basically copy the code from above with the following tweaks:
#  * replace "a" with a(v[n]) where "n" refers to the appropriate current moment
#  * get rid of the stuff that plots the analytic x and v, since we don't know them now!


def makegraph(Nsteps,tfinal):

  Dt=tfinal/(Nsteps)

  # Define constants and initial values
  x_0 = 0.0   # initial value of x
  v_0 = 5.0   # initial velocity in m/s

  # Create arrays to fill in
  t = np.zeros(Nsteps+1)
  x = np.zeros(Nsteps+1)
  v = np.zeros(Nsteps+1)

  # Fill in initial values
  x[0] = x_0
  v[0] = v_0

  # Now do all the steps using a loop:
  for n in range(Nsteps):
    t[n+1] = t[n] + Dt
    x[n+1] = x[n] + v[n]*Dt     # update rule for y:  same as before...
    v[n+1] = v[n] + a(v[n])*Dt        # update rule for v ---   THIS IS WHERE I REPLACED "a" WITH "a(v[n]" !

  y = np.zeros(Nsteps+1)  #set all y coordinates to zero (we'll use this again in the plot...)

  # and now make a multi-part plot...
  plt.figure(figsize=(14,7))
  gridspec.GridSpec(4,2)

  # motion diagram figure
  plt.subplot2grid((4,2),(0,0),colspan=2)
  plt.plot(x,y,'k.')        # plot x -- the list of x coordinates we generated above -- vs. zero
  plt.title("Positions...")  # give the plot a title
  plt.xlabel("x (in meters)")  # label for the horizontal axis
  plt.ylabel("y (in meters)")  # label for the vertical axis
  plt.axis([-.2,max(x)+.2,-.2,.2])   # set the range/scale on the x and y axes
  #plt.show()                   # draw the plot

  # x vs t graph
  plt.subplot2grid((4,2),(2,0),rowspan=2)
  plt.plot(t,x,'b.',label="numerical")          # plot x -- the list of x coordinates we generated above -- vs. zero
  plt.title("x vs t")  # give the plot a title
  plt.xlabel("t (in sec)")  # label for the horizontal axis
  plt.ylabel("x (in meters)")  # label for the vertical axis
  plt.legend()
  #plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
  #plt.show()                   # draw the plot


  # v vs t graph
  plt.subplot2grid((4,2),(2,1),rowspan=2)
  plt.plot(t,v,'g.',label="numerical")          # plot x -- the list of x coordinates we generated above -- vs. zero
  plt.title("v vs t")  # give the plot a title
  plt.xlabel("t (in sec)")  # label for the horizontal axis
  plt.ylabel("v (in m/s)")  # label for the vertical axis
  plt.legend()
  #plt.axis([-.2,1.2,-.2,.2])  # if we don't ask for specific axis ranges, it'll just default to show everything
  #plt.show()                   # draw the plot

  plt.show()


# Now create an interactive widget with slider bars to give input to the function makegraph

interact(makegraph,
         Nsteps=IntSlider(min=1, max=1000, step=1, value=8, layout=Layout(width='80%')),
         tfinal=FloatSlider(min=0.1, max=20, step=.1, value=10, readout_format='.1f', layout=Layout(width='80%'))
        );

# Google doc

Open up a google doc with your group to document your work on this part.  Include in your doc:

* The names of everybody in your group

* A quick summary of your explanation from before of why the algorithm gets more accurate as the number of time steps gets bigger (i.e., as the duration Dt of each step gets smaller)

* A picture of your by-hand-at-the-white-boards predictions for what the motion diagram, x-vs-t graph, and v-vs-t graph will look like here

* A screenshot of what the numerical simulation yields for a value of Nsteps that you think makes it pretty accurate.  (Please also explain briefly how you decided on that value of Nsteps!)

* A brief discussion comparing your by-hand predictions to the numerical simulation.  It doesn't matter if your prediction was right, but if it wasn't, you should try to understand/explain what went wrong.


Share your doc with your instructor so they can look through it later.  Note that this doesn't need to be a formal lab report, so don't spend too much time making it look nice.  We just want to see some record of your thoughtful engagement with this project.



# 6. One final challenge...

Let's consider one last example.  Suppose, as in the "cart with a sail" example just above, an object starts at $x=0$ with initial velocity $v_0 = 5 \, m/s$ (in the positive x direction).  But suppose now its acceleration $a$ is proportional to the position $x$, again with a negative proportionality constant.  That is, suppose

$$ a = - m x $$

with, say, $m = 0.5 \, s^{-2}$.

How will the object move?  And what kind of real-life situation might give rise to this kind of behavior?  Try first to figure out qualitatively what will happen by drawing/talking/thinking at the whiteboards.  Then, create a new code cell below, copy the code from the previous code cell into it, and see if you can figure out how to tweak it to address this new problem.  

(Instructors / LAs will be happy to try to help if/when you run into coding snafus...)

Add your results from this "final challenge" to your blog post, but if you don't have time to finish this completely, that's OK.  We're going to revisit this topic (using this kind of computer algorithm to figure out how things will move, when the equations of motion are too hard to solve analytically) a couple more times later in the course and really the main goal for today is just to get you used to the idea of using the computer to solve (step by step) for the trajectory of an object given initial values for its position and velocity and some kind of rule that determines what the acceleration will be.

So if you feel like that goal has been achieved, we'll declare victory and call it a day!
